In [1]:
!pip install openai

In [2]:
from openai import OpenAI
import numpy as np
import pandas as pd
import os

from google.colab import userdata
api_key = userdata.get('sutaaka')

if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("APIキーが見つかりません。")

client = OpenAI()

In [3]:
def vectorize_texts(text):
    """ 文書リストをベクトル化して返します """
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return np.array(response.data[0].embedding).reshape(1, -1)

In [4]:
def find_most_similar(question_vector, vectors, documents):
    """ 類似度が最も高い文書を見つけます """
    vectors = np.vstack(vectors)  # ドキュメントベクトルを垂直に積み重ねて2次元配列にする
    similarities = np.dot(vectors, question_vector.T).flatten() / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(question_vector))
    print("similarities", similarities)
    most_similar_index = np.argmax(similarities)
    return documents[most_similar_index]

In [5]:
def ask_question(question, context):
    """ GPTを使って質問に答えます """
    prompt = f'''以下の質問に以下の情報をベースにして答えてください。
            [ユーザーの質問]
            {question}

            [情報]
            {context}
            '''
    print("prompt", prompt)
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=200
    )
    return response.choices[0].text

In [6]:
# 情報
documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]

# 文書をベクトル化
vectors = [vectorize_texts(doc) for doc in documents]

# 質問
question = "2023年の第1事業部の売上ってどのくらい？"

# 質問をベクトル化
question_vector = vectorize_texts(question)

# 最も類似した文書を見つける
similar_document = find_most_similar(question_vector, vectors, documents)

# GPTモデルに質問
answer = ask_question(question, similar_document)
print(answer)

similarities [0.60348213 0.85603513 0.62467734]
prompt 以下の質問に以下の情報をベースにして答えてください。
            [ユーザーの質問]
            2023年の第1事業部の売上ってどのくらい？

            [情報]
            2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円
            



            2023年の第1事業部は300億円の売上である。
            


In [7]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity


url = "https://toukei-lab.com/achademy/?page_id=1619"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text_nodes = soup.find_all("div")

joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)